In [ ]:
import matplotlib.pyplot as pp
import numpy as np
import scipy as sp
from scipy import sparse
import time
%matplotlib ipympl

# Metropolis Simulation of the 2D Ising Model

In [ ]:
def gen_metropolis(metropolis_step):
    def metropolis_iterate(x0, num_steps, *args, **kwargs):
        '''Iterate metropolis algorithm for num_steps using iniital position x_0'''

        for n in range(num_steps):
            if n == 0:
                x = x0
            else:
                x = metropolis_step(x, *args, **kwargs)
            yield x
            
    return metropolis_iterate

def gen_prob(beta=1):
    def prob(dE):
        return np.where(dE < 0, 1, np.exp(-beta * dE))
    return prob

## Full interaction energy for random site changes

In this scheme, we generate a metropolis step by choosing a fraction of lattice sites to simultaneaously reverse the spin on. 

The metropolis criteria is then applied on the full interaction energy calculation of the entire grid.

### The functions

In [ ]:
X, Y = np.mgrid[-5:6, -5:6]

conv = np.exp(-(X**2 + Y**2) / 2.5**2 )

def interaction(arr):
    result = sp.signal.convolve2d(arr, conv, mode='same')

    result = result * arr
    return -1 * result.sum()

def gen_g(arr, prob=0.01):
    M, N = arr.shape
    def g():
        '''Random step vector.'''
        step = np.random.rand(M, N)
        
        step = np.floor(step + prob)
        
        step = -2 * step + 1
        return step
    return g

def metropolis_step(x, g, f):
    '''Perform one full iteration and return new position.'''
    x_proposed = x * g()
    
    dE = interaction(x_proposed) - interaction(x)
    if dE < 0:
        #print('1', end='')
        return x_proposed
    
    a = f(dE)
    
    if np.random.choice([True, False], p=[a, 1-a]):
        x_new = x_proposed
        #print('1', end='')
    else:
        x_new = x
        #print('0', end='')

    return x_new

metropolis_iterate = gen_metropolis(metropolis_step)

### Test

In [ ]:
M = N = 40
arr = np.random.rand(M, N)
arr = 2 * np.floor(2 * arr) - 1

fig, axes = pp.subplots(1,2)
axes[0].imshow(arr)
ax = axes[1]
im = ax.imshow(arr)

In [ ]:
g = gen_g(arr, prob=0.005)
prob = gen_prob(beta=4)

metarr = metropolis_iterate(arr,  1000, g, prob)
for newarr in metarr:
    im.set_data(newarr)
    fig.canvas.draw()

## All sites, individual energy contribution only

### Functions

In [ ]:
X, Y = np.mgrid[-5:6, -5:6]

conv = np.where(((X == Y) & (X == 0)), 0, np.exp(-(X**2 + Y**2) / 2.5**2 ))

def interaction(arr):
    '''
    Returns array of energies for each site.
    Negative values are favourable.
    '''
    
    result = sp.signal.convolve2d(arr, conv, mode='same')

    result = result * arr
    return -1 * result



def metropolis_step(x, f):
    '''Perform one full iteration and return new position.'''
    M, N = x.shape
    
    dE = -1 * interaction(x) # energies of flipping
    
    prob = f(dE) # probabilities of accepting
    
    a = np.random.random((M, N))
    
    return np.where(a < prob, -x, x)
    
    
metropolis_iterate = gen_metropolis(metropolis_step)   

### Test

In [ ]:
M = N = 500
arr = np.random.rand(M, N)
arr = 2 * np.floor(2 * arr) - 1

fig, axes = pp.subplots(1,2)
fig.set_size_inches(9,5)
fig.tight_layout()

axes[0].imshow(arr)
ax = axes[1]
im = ax.imshow(arr)
print(arr.sum())

In [ ]:
prob = gen_prob(beta=0.2)
metarr = metropolis_iterate(arr,  200, prob)
for newarr in metarr:
    im.set_data(newarr)
    #time.sleep(0.5)
    fig.canvas.draw()